In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_10ind = df.iloc[:1171].copy()
df_10ind['Date'] = pd.to_datetime(df_10ind['Date'], format='%Y%m')
df_10ind.set_index('Date', inplace=True)
df_10ind = df_10ind.apply(pd.to_numeric, errors='coerce')

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')

df_marketCap = df_numfirm * df_avgsize

In [4]:
df_marketCap

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,2842.20,1962.85,4888.23,4314.09,1615.50,1751.80,1606.79,178.64,1705.62,5835.32
1926-08-01,2880.00,2258.85,5101.59,4257.99,1661.40,1766.35,1604.33,180.60,1823.01,5942.67
1926-09-01,2986.20,2328.78,5227.32,4313.50,1700.82,1804.80,1589.16,187.11,1785.21,6178.84
1926-10-01,2998.80,2432.75,5256.53,4147.00,1680.84,1820.80,1589.57,188.09,1814.61,6168.67
1926-11-01,2954.70,2226.66,5050.79,4109.50,1602.90,1818.70,1549.39,185.78,1762.32,5977.70
...,...,...,...,...,...,...,...,...,...,...
2023-09-01,1659554.24,1155109.68,3509781.62,1721237.10,16047043.56,983910.48,4470108.23,4208050.80,1198434.88,9243217.60
2023-10-01,1576492.32,1125106.64,3256265.67,1773725.80,14998661.86,952148.96,4214974.64,4005456.87,1135025.80,8914129.38
2023-11-01,1515562.50,923313.84,3159303.75,1655706.96,14635091.52,944421.52,4230248.16,3814201.93,1145836.04,8678221.20


In [1]:
import pandas as pd

# Créer deux DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [7, 8, 9], 'B': [10, 11, 12]})

# Multiplication des deux DataFrames
result = df1.mul(df2)

print(result)


    A   B
0   7  40
1  16  55
2  27  72


In [20]:
def rolling_pf_SR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()



In [21]:
def rolling_pf_SR_NS(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [22]:
def rolling_pf_VAR(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        var = window.var()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / var) / np.sum(1 / var)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [23]:
def rolling_pf_VOL(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        std = window.std()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / std) / np.sum(1 / std)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [24]:
def rolling_pf_SameWeights(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        weights = (1 / window.shape[1])
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [25]:
#def rolling_pf_MarketCap(returns, window_size):

In [26]:
def rolling_pf_minVAR(returns, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def min_volatility(w):
            w = np.array(w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            return V

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(min_volatility, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()